Running a simulation with SHyFT
=========

### This notebook is guiding through the simulation process of a catchment. The following steps are described:
1. **Loading required python modules and setting path to SHyFT installation**
2. **Configuration of a SHyFT simulation**
3. **Running a SHyFT simulation**
4. **Post-processing:** Fetching simulation results from the simulator-object.

In [ ]:
# first you should import the third-party python modules which you'll use later on
# the first line enables that figures are shown inline, directly in the notebook
%pylab inline
import os
from os import path
import sys
from matplotlib import pyplot as plt

## 1. Loading required python modules and setting path to SHyFT installation

In [ ]:
# set the path for your shyft build
# this should point to the directory that is created
# when you clone shyft, assuming you have built shyft
# there and not installed it to your system python
shyft_path = os.path.abspath("../../../shyft")
sys.path.insert(0, shyft_path)

# you could achieve the same by setting a PYTHONPATH

In [ ]:
# once the shyft_path is set correctly, you should be able to import shyft modules
import shyft
from shyft import shyftdata_dir

# if you have problems here, it may be related to having your LD_LIBRARY_PATH
# pointing to the appropriate libboost_python libraries (.so files)
from shyft.repository.default_state_repository import DefaultStateRepository
from shyft.orchestration.configuration import yaml_configs
from shyft.orchestration.simulators.config_simulator import ConfigSimulator
from shyft import api

## 2. Configuration of a SHyFT simulation

In [ ]:
# now you can access the api of shyft with tab completion and help, try this:

#help(api.GeoPoint) # remove the hashtag and run the cell to print the documentation of the api.GeoPoint class
#api. # remove the hashtag, set the pointer behind the dot and use 
      # tab completion to see the available attributes of the shyft api

In [ ]:
# set up configuration using *.yaml configuration files
# here is the *.yaml file that configures the simulation:
config_file_path = os.path.abspath("./sagelva-config/sagelva_hbv/sagelva_simulation.yaml")

# and here we pass it to the configurator, together with the name of the region 
# stated in the simulation.yaml file (here: "neanidelva") which we would like to run
cfg = yaml_configs.YAMLSimConfig(config_file_path, "sagelva")

In [ ]:
# Once we have all the configuration in place (read in from the .yaml files)
# we can start to do the simulation. Here we use the ConfigSimulator class 
# to initialize a simulator-object. Shyft has several ways to achieve this
# but the .yaml configs are the most straight forward

simulator = ConfigSimulator(cfg)

In [ ]:
# Now the simulator is ready to run! You can explore this object with tab
# completion. As an example, you can see here how to get the number of cells
# in the region that was set up. This is used later for extracting the data.

n_cells = simulator.region_model.size()

In [ ]:
# We need to instantiate a state_repos repository from the DefaultStateRepository class
# this will be used to provide initial values for the state variables of the simulation
state_repos = DefaultStateRepository(simulator.region_model.__class__, n_cells)

## 3. Running a SHyFT simulation

In [ ]:
# To run a simulation, we need a time_axis (length of the simulation), and
# an initial state. We get the time_axis from the cfg object (which takes it
# from the .yaml files) and the state we constructed above.

simulator.run(cfg.time_axis, state_repos.get_state(0))

## 4. Post processing
####Examples for fetching data

####a) Get the discharge for each (sub-) catchment

In [ ]:
# Here we are going to extact data from the simulator object.
# We start by creating a list to hold discharge for each of the subcatchments.
# Then we'll get the data from the simulator object

# mapping of internal catch ID to catchment
catchment_id_map = simulator.region_model.catchment_id_map 

# api.TsVector() is a list of api.Timeseries type. 
discharge_ts = api.TsVector() #  except from the type, it just works as a list()
# loop over each catchment, and extract the time-series (we keep them as such for now)
for cid in catchment_id_map: # fill in discharge time series for all subcatchments
    discharge_ts.append(simulator.region_model.statistics.discharge([int(cid)]))
    # discharge is a TS object,keeping a .time_axis and .values
    # we can always do .values.to_numpy() to get the numpy array of the values.

In [ ]:
# We can make a quick plot of the data of each sub-catchment
fig, ax = plt.subplots(figsize=(20,15))

# plot each catchment discharge in the catchment_id_map
for i,ts in enumerate(discharge_ts):
    # a ts.time_axis can be enumerated to it's UtcPeriod, that have a .start and .end of type utctimestamp
    # since matplotlib do have built in support for datetime-axis, we convert it to datetime
    ts_timestamps = [datetime.datetime.utcfromtimestamp(p.start) for p in ts.time_axis]
    ts_values = ts.values # iterable and convertible, .to_numpy() makes an np array
    ax.plot(ts_timestamps,ts_values, label = "{}".format(catchment_id_map[i]))
fig.autofmt_xdate()
ax.legend(title="Catch. ID")
ax.set_ylabel("discharge [m3 s-1]")

In [ ]:
#
# now we can also plot the statistical distribution of the discharges over the sub-catchments
#
percentiles= api.IntVector([10,25,50,-1,75,90])  # the percentiles we want, note -1 = arithmetic average

# create a Daily(for the fun of it!) time-axis for the percentile calculations(our simulation could be hourly)
ta_statistics = api.Timeaxis2(simulator.region_model.time_axis.time(0),api.Calendar.DAY,365)

# then simply get out a new set of time-series, corresponding to the percentiles we specified
discharge_percentiles = api.percentiles(discharge_ts,ta_statistics,percentiles)

#utilize that we know that all the percentile time-series share a common time-axis
common_timestamps = [datetime.datetime.utcfromtimestamp(p.start) for p in ta_statistics] 

# Then we can make another plot of the percentile data for the sub-catchments
fig, ax = plt.subplots(figsize=(20,15))

# plot each discharge percentile in the discharge_percentiles
for i,ts_percentile in enumerate(discharge_percentiles):
    clr='k'
    if percentiles[i] >= 0.0: 
        clr= str(float(percentiles[i]/100.0))
    ax.plot(common_timestamps, ts_percentile.values, label = "{}".format(percentiles[i]),color=clr)

# also plot catchment discharge along with the statistics
# notice that we use .average(ta_statistics) to properly align true-average values to time-axis
ax.plot(common_timestamps,discharge_ts[0].average(ta_statistics).values,label = "CID {}".format(catchment_id_map[0]),linewidth=2.0,alpha=0.7,color='b')

fig.autofmt_xdate()
ax.legend(title="Percentiles")
ax.set_ylabel("discharge [m3 s-1]")

In [ ]:
# a simple percentile plot, from orchestration looks nicer
from shyft.orchestration import plotting as splt
oslo=api.Calendar('Europe/Oslo')
fig,ax=plt.subplots(figsize=(16,8))
splt.set_calendar_formatter(oslo)
h,ph=splt.plot_np_percentiles(common_timestamps,[ p.values.to_numpy() for p in discharge_percentiles],base_color=(0.03,0.01,0.3))

In [ ]:
# If we're interested to look at values of individual cells, rather
# than at the catchment level, we can do that also. Shyft does not have
# an underlying 'raster' model, so you need to fetch all cells directlry
# from the simulator object
cells = simulator.region_model.get_cells()

# Once we have the cells, we can get their coordinate information
# and fetch the x- and y-location of the cells
x = [cell.geo.mid_point().x for cell in cells]
y = [cell.geo.mid_point().y for cell in cells]


# If you want to know the membership to each catchment
# of the cells, you can get a list of catchment ids for each cell
cid_z_map = dict([ (catchment_id_map[i],i) for i in range(len(catchment_id_map))])
catch_ids = np.array([cid_z_map[cell.geo.catchment_id()] for cell in cells])

# and make a quick catchment map...
# using a scatter plot of the cells
fig, ax = plt.subplots(figsize=(15,5))
cm = plt.cm.get_cmap('rainbow')
plot = ax.scatter(x, y, c=catch_ids, marker='.', s=40, lw=0, cmap=cm)
plt.colorbar(plot).set_label('zero-based mapping(proper map tbd)')

####b) Get the Snow Cover Fraction (SCF) of all cells for a certain point in time

In [ ]:

# Here we'll do some more work to look at the timeseries data in each of the cells
# This example is collecting the response variable (here the SCF)
# for each of the cells for a certain point of time.
# set a date: year, month, day, (hour of day if hourly time step)
oslo = api.Calendar('Europe/Oslo') # specifying input calendar in olson tz-id
time_x = oslo.time(1991,1,15)  # the oslo calendar(incl dst) converts calendar coordinates Y,M,D.. to its utc-time

try:
    idx = simulator.region_model.time_axis.index_of(time_x) # index of time x on time-axis
except:
    print("Date out of range, setting index to 0")
    idx = 0

# fetching SCF (the response variable is named "snow_sca")
# You can use tab-completion to explore the `rc`, short for "response collector"
# object of the cell, to see further response variables available.
scf = simulator.region_model.hbv_snow_response.outflow([],idx) # specifying empty list [] indicates all catchments

In [ ]:
# And finally we can make a simple scatter plot for quick visualization
fig, ax = plt.subplots(figsize=(15,5))
cm = plt.cm.get_cmap('summer')
plot = ax.scatter(x, y, c=scf, vmin=0, vmax=1, marker='s', s=40, lw=0, cmap=cm)
plt.colorbar(plot)
plt.title('Snow Covered area of {0} on {1}'.format(cfg.region_model_id, oslo.to_string(time_x)))



In [ ]:
# And let's compute histogram of the snow covered area as well
fig, ax = plt.subplots(figsize=(12,8))
ax.hist(scf, bins=20, range=(0,1), color='y', alpha=0.5)
ax.set_xlabel("SCF of grid cell")
ax.set_ylabel("frequency")